In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import Hyperband
from tensorflow import keras
import pickle
import numpy as np

In [ ]:
pickle_in = open("X_audio.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y_audio.pickle","rb")
y = pickle.load(pickle_in)

X = X[..., np.newaxis]

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        
        model = Sequential()
        
        # Flatten
        model.add(Flatten())
        
        # Hidden layer 1
        model.add(
            Dense(
                units=hp.Int(
                    'units_1',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_1',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        # Dropout 1
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_1',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 2
        model.add(
            Dense(
                units=hp.Int(
                    'units_2',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_2',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        # Dropout 2
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_2',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 3
        model.add(
            Dense(
                units=hp.Int(
                    'units_3',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_3',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        # Dropout 4
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_3',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 4
        model.add(
            Dense(
                units=hp.Int(
                    'units_4',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_4',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        # Dropout 5
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_4',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )

        model.add(Dense(self.num_classes, activation='softmax'))
        
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
NUM_CLASSES = 3  # happy neutral sad number of classes
INPUT_SHAPE = (X.shape[1], X.shape[2], X.shape[3]) 
SEED = 1
HYPERBAND_MAX_EPOCHS = 15
MAX_TRIALS = 40
EXECUTION_PER_TRIAL = 2

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search',
    project_name='audioM3'
)

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='audioM3'
)

N_EPOCH_SEARCH = 15
BATCH_SIZE = 16

tuner.search(X, y, batch_size=BATCH_SIZE, epochs=N_EPOCH_SEARCH, validation_split=0.2)

tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

Trial 29 Complete [00h 00m 20s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.11969696870073676
Total elapsed time: 00h 04m 11s

Search: Running Trial #30

Hyperparameter    |Value             |Best Value So Far 
units_1           |96                |96                
dense_activation_1|relu              |relu              
dropout_1         |0.2               |0.2               
units_2           |480               |288               
dense_activation_2|tanh              |relu              
dropout_2         |0.2               |0.35              
units_3           |192               |480               
dense_activation_3|relu              |relu              
dropout_3         |0.35              |0.5               
units_4           |384               |480               
dense_activation_4|relu              |sigmoid           
dropout_4         |0.3               |0.2               
learning_rate     |0.001             |0.0001            
tuner/epochs      |15                |15     